In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: benellis3. Use `wandb login --relogin` to force relogin


True

In [2]:
import wandb
import os
import uuid
from pathlib import Path

sweep = "cmwbqphs"
# Download checkpoint from wandb
api = wandb.Api()
ckpt_dir = Path("./craftax_checkpoints")
ckpt_runs = api.sweep(f"oxwhirl/craftax-ppo-tuning/cmwbqphs").runs
for ckpt_run in ckpt_runs:
    num_train_interactions = ckpt_run.config["num_train_interactions"]
    reset_type = ckpt_run.config["ppo_reset_on_batch"]
    rand_end_string = str(uuid.uuid4())[:6]
    name = f"craftax_reset_type={reset_type}_num_train_interactions={num_train_interactions}_{rand_end_string}"
    for file in ckpt_run.files():
        file.download(root=(ckpt_dir / ckpt_run.name), exist_ok=True)

In [13]:
from pprint import pprint

run = ckpt_runs[3]
pprint(run.name)

'magic-sweep-15'


In [15]:
import jax
from util import *
from experiments.parse_args import parse_args
from agents.agents import get_agent
from environments.rollout import RolloutWrapper

cmd_args = "--agent=ppo --env_name=Craftax-Symbolic-v1 --gae_lambda=0.9 --layer_width=256 --lr=0.0003 --max_grad_norm=1 --num_agents=1 --num_env_workers=512 --num_minibatches=4 --num_rollout_steps=64 --num_train_interactions=32768 --ppo_num_epochs=4 --save_policy --seed=2".split(
    " "
)
args = parse_args(cmd_args)

rng = jax.random.PRNGKey(args.seed)
env = RolloutWrapper(args.env_name, args.num_rollout_steps)
env_params = env.default_env_params
rng, _rng = jax.random.split(rng)
_rng = jax.random.split(_rng, args.num_env_workers)
obsv, env_state = env.batch_reset(_rng, env_params)
train_state, aux_train_states, agent_train_step_fn = get_agent(
    args,
    rng,
    env.obs_shape,
    env.num_actions,
    env.discrete_actions,
    env.action_lims,
)
train_state

TrainState(step=0, apply_fn=<bound method Module.apply of ActorCritic(
    # attributes
    width = 256
    num_actions = 17
    activation = 'relu'
)>, params={'params': {'actor_0': {'kernel': Array([[ 0.00392421,  0.02899976, -0.01661767, ..., -0.02550365,
         0.09271209,  0.00108168],
       [ 0.02584682,  0.04291209, -0.02767812, ...,  0.00776166,
         0.00491402,  0.12317209],
       [-0.00770354, -0.03328874, -0.04437829, ..., -0.00889074,
         0.00904444,  0.02584643],
       ...,
       [ 0.04543079,  0.04821817, -0.01019958, ...,  0.027421  ,
        -0.01652528,  0.00430968],
       [-0.06850644, -0.00809783, -0.00156537, ...,  0.02846765,
         0.01694525, -0.02046602],
       [ 0.00968764,  0.0542812 ,  0.0134697 , ..., -0.02364796,
        -0.02413787, -0.02466886]], dtype=float32), 'bias': Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 

In [12]:
def train(rng, env, args, train_state):
    # --- Initialize environment ---
    env_params = env.default_env_params
    rng, _rng = jax.random.split(rng)
    _rng = jax.random.split(_rng, args.num_env_workers)
    obsv, env_state = env.batch_reset(_rng, env_params)

    # --- Initialize agent train states and step function ---
    rng, _rng = jax.random.split(rng)
    # train_state contains actor (and critic if used) to be used for rollouts,
    # aux_train_states contains all other trainable parameters
    _, aux_train_states, agent_train_step_fn = get_agent(
        args,
        _rng,
        env.obs_shape,
        env.num_actions,
        env.discrete_actions,
        env.action_lims,
    )

    # --- Execute train loop ---
    def _train_step(runner_state, _):
        train_state, aux_train_states, env_state, last_obs, rng = runner_state
        # --- Collect trajectories ---
        rng, _rng = jax.random.split(rng)
        _rng = jax.random.split(_rng, args.num_env_workers)
        rollout = env.batch_rollout(_rng, train_state, env_params, last_obs, env_state)
        if args.log_dormancy:
            new_env_state, new_last_obs, traj_batch, dormancy = rollout
        else:
            new_env_state, new_last_obs, traj_batch = rollout

        # --- Update agent ---
        rng, _rng = jax.random.split(rng)
        train_state, aux_train_states, loss, metric = agent_train_step_fn(
            train_state, aux_train_states, traj_batch, new_last_obs, _rng
        )

        # --- Aggregate metrics ---
        if args.log_dormancy:
            metric["dormancy"] = dormancy
        # Manually aggregate Craftax achievements - NaN when no episodes end
        metric.update(
            {
                k: (v * traj_batch.done).sum() / traj_batch.done.sum()
                for k, v in metric.items()
                if "achievements" in k.lower()
            }
        )
        # metric, loss = jax.tree_map(jnp.mean, (metric, loss))

        runner_state = (
            train_state,
            aux_train_states,
            new_env_state,
            new_last_obs,
            rng,
        )
        return runner_state, (loss, metric)

    rng, _rng = jax.random.split(rng)
    runner_state = (train_state, aux_train_states, env_state, obsv, _rng)
    runner_state, (loss, metric) = jax.lax.scan(
        _train_step, runner_state, None, args.num_train_iters
    )
    ret = {
        "runner_state": runner_state,
        "metrics": metric,
        "loss": loss,
    }
    if args.save_policy:
        ret["policy"] = train_state
    return ret

In [13]:
from orbax.checkpoint import PyTreeCheckpointer
import jax

from util import *
from experiments.parse_args import parse_args
from agents.agents import get_agent
from environments.rollout import RolloutWrapper
from omegaconf import OmegaConf
from tqdm import tqdm
import wandb
from pathlib import Path
from functools import partial

# Create placeholder train state
# TODO: This
# Restore checkpoint into placeholder train state
sweep = "cmwbqphs"
api = wandb.Api()
ckpt_dir = Path("./craftax_checkpoints")
ckpt_runs = api.sweep(f"oxwhirl/craftax-ppo-tuning/{sweep}").runs


def get_fake_train_state(run):
    args = OmegaConf.create(run.config)
    args.num_train_iters = 10
    env = RolloutWrapper(args.env_name, args.num_rollout_steps)
    env_params = env.default_env_params
    rng = jax.random.PRNGKey(args.seed)
    rng, _rng = jax.random.split(rng)
    _rng = jax.random.split(_rng, args.num_env_workers)
    obsv, env_state = env.batch_reset(_rng, env_params)
    train_state, aux_train_states, agent_train_step_fn = get_agent(
        args,
        rng,
        env.obs_shape,
        env.num_actions,
        env.discrete_actions,
        env.action_lims,
    )
    return train_state, env, args


def download_file(run):
    for file in run.files():
        file.download(root=(ckpt_dir / run.name), exist_ok=True)


rng = jax.random.PRNGKey(0)
metrics = {}
ckptr = PyTreeCheckpointer()
seeds = 3
for run in tqdm(ckpt_runs):
    download_file(run)
    fake_train_state, env, args = get_fake_train_state(run)
    path = ckpt_dir / run.name / "policy"
    if not path.exists():
        print(f"Skipping {path}")
        continue
    train_state = ckptr.restore(path, item=fake_train_state)
    rng, _rng = jax.random.split(rng)
    _rng = jax.random.split(_rng, num=seeds)
    train_fn = jax.vmap(train, in_axes=(0, None, None, None))
    ret = train_fn(_rng, env, args, train_state)
    metrics[str(run.name)] = ret["loss"]
    print(ret["loss"])

  6%|▌         | 1/18 [00:03<01:06,  3.92s/it]

Skipping craftax_checkpoints/atomic-sweep-18/policy


 11%|█         | 2/18 [00:07<01:01,  3.87s/it]

Skipping craftax_checkpoints/apricot-sweep-17/policy


 17%|█▋        | 3/18 [00:11<00:58,  3.93s/it]

...

{'actor_loss': Array([[[[[-3.60895932e-01,  4.88976032e-01, -3.18296365e-02, ...,
           -1.66139245e+00,  4.74831425e-02,  2.71667778e-01],
          [ 2.57787883e-01,  1.55277491e-01,  6.61398768e-01, ...,
           -2.11500786e-02,  4.10615504e-01, -9.54905748e-01],
          [ 1.75683469e-01, -6.48424804e-01, -2.48210764e+00, ...,
           -6.46337152e-01,  7.37519860e-01,  2.23923013e-01],
          [ 6.08945429e-01, -2.08109331e+00, -1.83250141e+00, ...,
           -3.18731874e-01,  7.56708503e-01, -7.69089013e-02]],

         [[ 3.89934063e-01, -5.28808296e-01, -2.26109576e+00, ...,
           -3.31898123e-01, -2.12483048e+00,  6.36451602e-01],
          [ 3.71839702e-01, -1.08166969e+00,  5.44035077e-01, ...,
            4.33940798e-01,  8.28038454e-01,  5.07082939e-01],
          [ 8.03725541e-01,  2.81008840e-01,  2.31807962e-01, ...,
           -8.95177364e-01,  2.64088511e-01,  3.33942205e-01],
          [ 5.53245187e-01,  7.47723699e-01,  3.85660888e-03, ...,
      

 94%|█████████▍| 17/18 [18:47<01:19, 79.57s/it]

{'actor_loss': Array([[[[[ 1.66228473e-01, -2.85035181e+00,  1.56910270e-01, ...,
            6.39105082e-01,  7.24125504e-01, -1.51709199e+00],
          [ 9.78314877e-02,  7.40326941e-02,  5.74040532e-01, ...,
           -1.33486748e+00,  1.47208989e+00,  5.44266582e-01],
          [ 7.51416922e-01,  5.44763863e-01,  5.79736650e-01, ...,
            4.64326590e-01, -1.50172257e+00, -1.11597776e+00],
          [ 3.74930710e-01,  5.01862168e-01,  9.38142896e-01, ...,
            5.42445362e-01,  4.65394855e-01,  4.35409367e-01]],

         [[ 9.63997424e-01, -3.18467259e-01,  3.01550359e-01, ...,
           -2.95993805e+00, -1.26410067e+00, -7.54270792e-01],
          [ 8.55333209e-02,  8.53828117e-02, -1.80014707e-02, ...,
            3.47356856e-01,  6.41704917e-01,  4.79236305e-01],
          [-2.77548933e+00, -3.12181258e+00, -5.67323864e-01, ...,
           -3.47532034e-01,  9.73343015e-01,  1.58555433e-01],
          [ 7.66500592e-01,  4.92574751e-01,  6.83853805e-01, ...,
      

100%|██████████| 18/18 [20:05<00:00, 66.98s/it]

{'actor_loss': Array([[[[[ 2.58066416e-01,  3.76827598e-01, -2.19447041e+00, ...,
            5.01944184e-01,  1.18804765e+00, -2.76163742e-02],
          [ 2.89486885e-01,  4.29829240e-01,  5.78206062e-01, ...,
           -5.70910722e-02,  4.04491246e-01, -8.02388564e-02],
          [-3.46968591e-01,  1.06353319e+00,  4.87039000e-01, ...,
            5.57751775e-01,  5.75671792e-01,  1.31307769e+00],
          [ 1.68243438e-01,  5.40759087e-01, -1.43020046e+00, ...,
            7.19032764e-01,  9.62851048e-01, -3.52476954e-01]],

         [[ 5.49538255e-01,  1.04060519e+00,  5.24648964e-01, ...,
            1.03638601e+00, -8.27159826e-03,  6.71506941e-01],
          [ 9.45902765e-01,  1.06055045e+00,  3.66713136e-01, ...,
            5.05317628e-01,  5.36502481e-01,  5.22158623e-01],
          [ 7.35092163e-01,  4.11176831e-01,  4.37734902e-01, ...,
            3.21274996e-01,  5.78827858e-01,  3.84271175e-01],
          [ 8.93521845e-01, -2.32418299e+00, -1.12056220e-03, ...,
      

In [17]:
import json
import numpy as np
jsonable_metrics = jax.tree_map(lambda x: np.array(x).tolist(), metrics)
with open("metrics.json", "w") as f:
    json.dump(jsonable_metrics, f)

AttributeError: 'ArrayImpl' object has no attribute 'asarray'

In [10]:
metrics["magic-sweep-15"]

{'actor_loss': Array([[-0.00448393, -0.00342102, -0.00421866, -0.00254218, -0.00430193,
         -0.0046559 , -0.00393579, -0.00609214, -0.0048873 , -0.00510402],
        [-0.00469506, -0.00287113, -0.0036848 , -0.00395979, -0.00344467,
         -0.00225743, -0.00326084, -0.00526722, -0.00560752, -0.00320093],
        [-0.00416508, -0.00402993, -0.00390111, -0.0027086 , -0.00248465,
         -0.00333057, -0.00332466, -0.00387943, -0.00476159, -0.00393472]],      dtype=float32),
 'entropy': Array([[2.8310232, 2.8146758, 2.7894177, 2.7690678, 2.7239418, 2.6993566,
         2.6514404, 2.58407  , 2.496291 , 2.4150963],
        [2.830948 , 2.8172991, 2.7965627, 2.7776713, 2.744974 , 2.725834 ,
         2.6810186, 2.6381903, 2.5429492, 2.498064 ],
        [2.8315773, 2.817326 , 2.8009045, 2.7676418, 2.7212412, 2.671113 ,
         2.6245584, 2.5464237, 2.4790277, 2.38649  ]], dtype=float32),
 'grad_norm': Array([[0.21600589, 0.17424749, 0.22911952, 0.24393004, 0.27448413,
         0.3186055 ,